In [57]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Imports
import os
import pickle
import pandas as pd
from datetime import datetime
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
import flModel
%matplotlib inline

In [58]:
DATA_FILE = 'data/model_vectors.csv'
df = pd.read_csv(DATA_FILE)
# df = df.dropna()
df

,Unnamed: 0,Body,Brand,FE-L100K-CMB,FE-L100K-CTY,FE-L100K-HWY,HP_value,Model,Price,Torque_value,Year,details,image,HP,Torque,Braking,RoadHolding,FuelEconomy
0,3,Sedan,Audi,10.46,11.77,9.11,252.0,A4,51800,273.0,2017,https://www.audi.ca/ca/web/en/models/a4/a4-sed...,https://en.wikipedia.org/wiki/Audi_A4#/media/F...,3.118653,3.741598,2.762206,3.790932,2.351097
1,4,Sedan,BMW,10.46,12.28,8.31,248.0,330i,45900,258.0,2017,https://www.bmw.ca/en/all-models/3-series.html,http://st.motortrend.ca/uploads/sites/42/2017/...,3.059861,3.421665,2.385011,2.482243,2.351097
2,5,Sedan,Infinity,11.30,12.28,9.42,208.0,Q50,39900,258.0,2018,http://www.infiniti.ca/en/sedan/q50?dcp=ppn.11...,http://www.infiniti.ca/content/dam/infiniti/ve...,2.477263,3.421665,5.000000,3.186398,2.176942
3,6,Sedan,Lexus,12.84,14.87,10.09,306.0,IS 300,41050,277.0,2017,https://www.lexus.ca/lexus/en/configure-price/is,https://s.aolcdn.com/dims-global/dims3/GLOB/le...,3.912346,3.826913,2.276601,2.581864,1.915709
4,7,Sedan,BMW,11.30,13.45,8.83,320.0,440i,56850,330.0,2018,https://www.bmw.ca/en/all-models/4-series/coup...,https://s1.cdn.autoevolution.com/images/news/2...,4.118119,4.957342,2.413746,2.984887,2.176942
5,8,Sedan,Mercedes,11.30,12.84,9.74,241.0,C300,44700,273.0,2018,https://www.mercedes-benz.ca/en/vehicles/class...,https://www.mercedes-benz.ca/content/dam/mb-na...,2.956975,3.741598,2.613020,4.596977,2.176942
6,10,Coupe,Audi,NaN,NaN,NaN,252.0,A5,46350,273.0,2018,https://www.audi.ca/ca/web/en/models/a5/a5-cou...,https://audi-encounter.com/media/audi-a7-sport...,3.118653,3.741598,3.259494,5.000000,2.351097
7,12,SUV,Volvo,12.28,14.13,10.46,316.0,XC90,59150,295.0,2018,https://www.volvocars.com/en-ca/cars/new-model...,https://assets.volvocars.com/uk/~/media/united...,4.059327,4.210832,2.399292,2.581864,2.002786
8,13,SUV,Lexus,12.84,9.74,10.87,295.0,RX 350,56500,267.0,2017,https://www.lexus.ca/lexus/en/automobiles/rx?m...,http://www.motortrend.com/cars/lexus/rx350/2016/#,3.750668,3.613625,4.353526,2.362617,1.915709
9,14,SUV,BMW,11.77,12.28,11.30,300.0,X5,69950,300.0,2016,https://www.bmw.ca/en/all-models/x-series/X5/2...,https://cars.usnews.com/static/images/Auto/izm...,3.824158,4.317477,2.428375,2.392523,2.089864


In [59]:
df.dtypes

Unnamed: 0        int64
Body             object
Brand            object
FE-L100K-CMB    float64
FE-L100K-CTY    float64
FE-L100K-HWY    float64
HP_value        float64
Model            object
Price             int64
Torque_value    float64
Year              int64
details          object
image            object
HP              float64
Torque          float64
Braking         float64
RoadHolding     float64
FuelEconomy     float64
dtype: object

In [60]:
def generate_user_vec(driving_log, web_map, m):
    run_df = m.format_df([driving_log])

    num_of_brakes_average, num_of_brakes_stepped = m.count_braking_steps(run_df)

    average_acc = run_df[run_df['acc'] > 0]['acc'].mean()
    average_rpm = run_df['rpm'].mean()
    average_vel = run_df['speed'].mean()
    average_throttle = run_df['throttle'].mean()
    average_lateral = run_df['lateral_velocity'].apply(abs).mean()


    run_df['deacc'] = -1 * run_df['acc'] 
    run_df = run_df.query('deacc > 1')

    average_deacc = run_df['deacc'].mean()

    brake_score = m.compute_brake(average_deacc, num_of_brakes_stepped)
    hp_score = m.compute_hp(average_acc, average_rpm)
    torque_score = m.compute_torque(average_vel, average_throttle)
    lateral_score = m.calculate_score(average_lateral, m.average_lateral_overall, m.max_lateral)
    
    performance = web_map.get('performance', 1)
    if performance == 2:
        torque_score = torque_score * 1.2
        hp_score = hp_score * 1.2
    elif performance == 3:
        torque_score = torque_score * 1.5
        hp_score = hp_score * 1.5
        
    performance = web_map.get('handling', 1)
    if performance == 2:
        torque_score = torque_score * 1.1
        lateral_score = lateral_score * 1.2
    elif performance == 3:
        torque_score = torque_score * 1.2
        lateral_score = lateral_score * 1.5

    torque_score = min(5, torque_score)
    hp_score = min(5, hp_score)
    lateral_score = min(5, lateral_score)
    
    cost = 0
    cost_pref = web_map.get('cost', 1)
    if cost_pref == 2:
        cost = 5.0
    elif cost_pref == 3:
        cost = 10.0
    
    fuel_economy = 1.0
    fuel_pref = web_map.get('fuel_economy', 1)
    if fuel_pref == 2:
        fuel_economy = 5.0
    elif fuel_pref == 3:
        fuel_economy = 10.0
    return [hp_score, torque_score, brake_score, lateral_score, fuel_economy, cost]

def norm_fuel(row, web_map):
#     return 1.0
    fuel_economy = 1.0
    fuel_pref = web_map.get('fuel_economy', 1)
    if fuel_pref == 2:
        fuel_economy = row.FuelEconomy
    elif fuel_pref == 3:
        fuel_economy = row.FuelEconomy * 2.0
    return fuel_economy

def norm_cost(row, web_map):
#     return 1.0
    cost = 1.0
    c = 6.25 * (75000 - row.Price)/ 75000
    cost_pref = web_map.get('cost', 1)
    if cost_pref == 2:
        cost = c
    elif cost_pref == 3:
        cost = c * 2
    return cost

def calc_rmse(row, user_vec):
    car_vector = [row.HP, row.Torque, row.Braking, row.RoadHolding, row.FuelEconomyNorm, row.PriceNorm]
    return sqrt(mean_squared_error(user_vec, car_vector))

def filter_cars(row, web_map=dict()):
    cond1 = row.Price > web_map.get('price_min', 0)
    cond2 = row.Price < web_map.get('price_max', 100000)
    cond3 = True
    fuel_pref = web_map.get('fuel_economy', 1)
    if fuel_pref == 2 and row.FuelEconomy < 1:
        cond3 = False
    elif fuel_pref == 3 and row.FuelEconomy < 2:
        cond3 = False
    cond4 = False
    if row['Body'] in set(web_map.get('body_types', ['Sedan'])):
        cond4 = True
    
    return cond1 & cond2 & cond3 & cond4

def find_car(models_file, driving_log, web_inputs, fuzzy_model):
    user_vector = generate_user_vec(driving_log, web_inputs, fuzzy_model)
    df = pd.read_csv(models_file).dropna()
    # Apply filtering
    df = df[df.apply(lambda x: filter_cars(x, web_inputs), axis=1)]
#     df[df.apply(lambda x: x['b'] > x['c'], axis=1)]
    df['FuelEconomyNorm'] = df.apply(lambda x: norm_fuel(x, web_inputs), axis=1)
    df['PriceNorm'] = df.apply(lambda x: norm_cost(x, web_inputs), axis=1)
    df['rmse_score'] = df.apply(lambda x: calc_rmse(x,user_vector), axis=1)
    df['percent_match'] = df.apply(lambda x: (5 - x.rmse_score) * 20, axis=1)
    rmse_norm_factor = df.rmse_score.min() / 2
    df['percent_match_norm'] = df.apply(lambda x: (5 - x.rmse_score + rmse_norm_factor) * 20, axis=1)
    return df.sort_values(by=['rmse_score'])

In [61]:
web_mapping = {
    'price_min': 10000,
    'price_max': 45000,
    'fuel_economy': 1, # 1,2,3
    'body_types': ['Sedan', 'Coupe']
}

with open('models/fuzzy_models.p', 'rb') as handle:
    fuzzy_models = pickle.load(handle)
    
model = find_car(DATA_FILE, 'hwd_data2/run_40.csv', web_mapping, fuzzy_models)
list_of_cars = model.T.to_dict().values()

In [62]:
find_car(DATA_FILE, None, None)

TypeError: find_car() missing 1 required positional argument: 'fuzzy_model'

In [63]:
def return_rmse(price, fuel_economy, data_file_name='hwd_data2/run_40.csv', car_body=['Sedan', 'Coupe']):
    
    web_mapping = {
        'price_min': 0,
        'price_max': price,
        'fuel_economy': 1, # 1,2,3
        'body_types': car_body,
        'performance': 1
    }

    with open('models/fuzzy_models.p', 'rb') as handle:
        fuzzy_models = pickle.load(handle)

    model = find_car(DATA_FILE, 'hwd_data2/run_40.csv', web_mapping, fuzzy_models)
    list_of_cars = model.T.to_dict().values()
    
    return model

In [64]:
return_rmse(30000, 3, car_body=['Sedan','Coupe'])

,Unnamed: 0,Body,Brand,FE-L100K-CMB,FE-L100K-CTY,FE-L100K-HWY,HP_value,Model,Price,Torque_value,...,HP,Torque,Braking,RoadHolding,FuelEconomy,FuelEconomyNorm,PriceNorm,rmse_score,percent_match,percent_match_norm
14,22,Sedan,Mazda,9.42,10.46,7.85,184.0,3,15900,185.0,...,2.191425,2.153287,2.428375,2.783375,2.583658,1.0,1.0,1.250750,74.984996,87.492498
16,24,Sedan,Kia,8.83,9.74,7.43,147.0,Forte,15495,132.0,...,1.750758,1.536400,4.154611,2.422430,2.713359,1.0,1.0,1.340125,73.197502,85.705004
23,33,Sedan,Subaru,10.87,12.28,9.11,152.0,Impreza,19995,145.0,...,1.810307,1.687712,2.356952,2.452336,2.264020,1.0,1.0,1.412626,71.747478,84.254980
17,25,Sedan,Hyundai,7.85,8.83,7.06,128.0,Elantra,15999,156.0,...,1.524469,1.815745,2.316080,2.392523,2.972763,1.0,1.0,1.430282,71.394368,83.901870
13,21,Coupe,Honda,8.56,9.42,7.24,180.0,Civic,19990,177.0,...,2.143785,2.060172,2.513562,4.395466,2.778210,1.0,1.0,1.627492,67.450150,79.957652
32,45,Coupe,Smart Fortwo,4.20,4.60,3.80,40.0,Pure,16700,73.0,...,0.476397,0.849676,4.502712,2.063551,5.000000,1.0,1.0,1.832491,63.350184,75.857686
